In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from functools import reduce

In [2]:
def read_data():
    data = Path('train_tickets.txt').read_text()
    sections = data.split("\n\n")
    
    ticket_fields = {}
    for line in sections[0].splitlines():
        field, ranges = line.split(":")
        ticket_fields[field] = []
        ranges = ranges.split(" or ")
        for r in ranges:
            low, high = r.split("-")
            ticket_fields[field].append((int(low),int(high)))
            
    my_ticket = [
        int(i) for i in sections[1].splitlines()[1].split(",")
    ]
    
    other_tickets = []
    for line in sections[2].splitlines()[1:]:
        other_tickets.append([
            int(i) for i in line.split(",") ])
            
    return ticket_fields, my_ticket, other_tickets

In [3]:
ticket_fields, my_ticket, other_tickets = read_data()

# Problem 1
Sorts tickets into valid and invalid tickets, sums the invalid entries 
in the invalid tickets

In [4]:
def flag_invalid(value, ranges):
    for r in ranges:
        if r[0] <= value <= r[1]:
            return 0
    return value
    
all_ranges = []
for items in ticket_fields.values():
    all_ranges.extend(items)
    
valid_tickets = []
total = 0
for ticket in other_tickets:
    ticket_total = 0
    for field in ticket:
        ticket_total += flag_invalid(field, all_ranges)
    if ticket_total:
        total += ticket_total
    else:
        valid_tickets.append(ticket)
total

26009

# Problem 2

Figure out which part of the ticket goes with which field and
multiply the departure fields together. 

In [5]:
arr = pd.DataFrame(np.array(valid_tickets))

In [6]:
num_tickets, num_fields = arr.shape
column_to_key = {
    c: [] for c in arr.columns
}

for column in arr.columns:
    for field in ticket_fields.keys():
        low1,high1 = ticket_fields[field][0]
        low2,high2 = ticket_fields[field][1]
        maybe =(((low1 <= arr[column]) & (arr[column] <= high1)) | 
                ((low2 <= arr[column]) & (arr[column] <= high2))).sum()
        if maybe == num_tickets:
            column_to_key[column].append(field)

In [7]:
cant_use = set()
while len(cant_use) < num_fields:
    for key,value in column_to_key.items():
        if len(value) == 1:
            cant_use.add(value[0])
        else:
            for v in value:
                if v in cant_use:
                    value.remove(v)
dep_keys = [
    k for k,v in column_to_key.items() if v[0].startswith('departure')
]
dep_vals = [my_ticket[i] for i in dep_keys]
prod = reduce(lambda a,b:a*b, dep_vals, 1)
prod

589685618167

In [8]:
column_to_key

{0: ['departure time'],
 1: ['zone'],
 2: ['arrival track'],
 3: ['departure date'],
 4: ['departure platform'],
 5: ['departure station'],
 6: ['wagon'],
 7: ['class'],
 8: ['arrival station'],
 9: ['route'],
 10: ['departure location'],
 11: ['duration'],
 12: ['departure track'],
 13: ['type'],
 14: ['arrival location'],
 15: ['row'],
 16: ['price'],
 17: ['seat'],
 18: ['arrival platform'],
 19: ['train']}